In [2]:
import io
import os
import torch
import warnings
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

warnings.filterwarnings('ignore')

C:\Users\Mr.wi\anaconda3\envs\nlp\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\Mr.wi\anaconda3\envs\nlp\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\Mr.wi\anaconda3\envs\nlp\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\Mr.wi\anaconda3\envs\nlp\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\Mr.wi\anaconda3\envs\nlp\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are de

In [3]:
import pandas as pd

In [22]:
import os

dir_path = "C:\\Users\\Mr.wi\\Desktop\mmm\\Machine-Learning\\dataset"
data_list = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        if '.csv' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_csv(file_path,encoding="cp949"))
        if '.xlsx' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_excel(file_path))

In [24]:
data_list[0]

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,anger,Angry,1,Angry,1,Neutral,0,Angry,1,Angry,1,27,male
3,5e25902f305bcf3ad153a6a9,그럼 방세는 어떡해.,anger,Sadness,1,Sadness,1,Sadness,1,Sadness,1,Sadness,1,27,male
4,5e27f90b5807b852d9e0157b,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad,Sadness,1,Sadness,1,Sadness,1,Sadness,2,Sadness,1,32,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,5e53d6067bef803b4851dfc6,"아, 요즘 룸메랑 너무 자주 싸우게 돼.",anger,Disgust,1,Angry,2,Angry,1,Disgust,1,Angry,1,35,male
14602,5e53d61cc38c123b9ec6dde6,"아, 룸메가 방을 너무 지저분하게 써. 음식물도 막 버리고.",anger,Disgust,1,Angry,2,Angry,1,Disgust,1,Angry,1,35,male
14603,5e53d6332a2d173b73a03210,"뭐 화를 낸 것까진 아니지만, 한 달 전 쯤에 좀 확실하게 얘기를 해뒀거든. 근데 ...",anger,Disgust,1,Angry,2,Sadness,1,Disgust,1,Angry,2,35,male
14604,5e53d659963e443aee02b7d0,"어. 고등학교 동창인데, 같은 동네 오게 돼서 같이 룸메로 살게 됐지.",anger,Neutral,0,Angry,1,Sadness,1,Neutral,0,Neutral,0,35,male


In [51]:
data = data_list[0].iloc[:,1:3]

for _ in range(1,3):
    data = pd.concat([data,data_list[_].iloc[:,1:3]])
buf = data_list[3].iloc[:,1:]
buf.columns = ['발화문',"상황"]
data = pd.concat([data,buf])

In [52]:
data

,발화문,상황
0,"어, 청소 니가 대신 해 줘!",anger
1,둘 다 청소 하기 싫어. 귀찮아.,anger
2,둘 다 하기 싫어서 화내.,anger
3,그럼 방세는 어떡해.,anger
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad
...,...,...
55623,얘긴 다 끝났냐? 원예부,중립
55624,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55625,여자 숨겨달라는거면 사절이다.,중립
55626,아무래도 안되나요?,중립
